<a href="https://colab.research.google.com/github/ferdouszislam/pytorch-practice/blob/main/pytorch_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import numpy as np

In [3]:
if torch.cuda.is_available():
  print('GPU, yay!')
else:
  print('CPU :(')

GPU, yay!


# Tutorial 01 - Tensor Basics

In [4]:
x = torch.rand(3,3)
y = torch.rand(3,3)

In [5]:
print(x, '\n', y, '\n')

tensor([[0.1219, 0.6695, 0.4641],
        [0.2987, 0.1864, 0.3337],
        [0.6971, 0.4852, 0.9293]]) 
 tensor([[0.8722, 0.0992, 0.1062],
        [0.3238, 0.0188, 0.2449],
        [0.5992, 0.6771, 0.5306]]) 



In [6]:
z = torch.add(x, y)
z

tensor([[0.9941, 0.7687, 0.5702],
        [0.6225, 0.2051, 0.5786],
        [1.2963, 1.1624, 1.4599]])

In [7]:
z = torch.mul(x, y)
z 

tensor([[0.1063, 0.0664, 0.0493],
        [0.0967, 0.0035, 0.0817],
        [0.4177, 0.3286, 0.4931]])

In [8]:
z.add_(y) # same as z+=y

tensor([[0.9785, 0.1656, 0.1554],
        [0.4205, 0.0222, 0.3266],
        [1.0168, 1.0057, 1.0237]])

In [9]:
z[:, 2] # get all rows at column 2 (0 based indexing)

tensor([0.1554, 0.3266, 1.0237])

In [10]:
z[1,2].item() # get single element value

0.32663848996162415

In [11]:
'''
input- a pytorch tensor variable 
returns- multiplication of the input tensor's dimensions 
'''
def get_flat_shape(tensor):
  dims=list(tensor.size())
  flat_dim = 1
  for dim in dims:
    flat_dim*=dim
  return flat_dim

In [12]:
get_flat_shape(z)

9

In [13]:
flat_z = z.view(get_flat_shape(z)) # resizing a tensor
flat_z

tensor([0.9785, 0.1656, 0.1554, 0.4205, 0.0222, 0.3266, 1.0168, 1.0057, 1.0237])

In [14]:
# tensor to numpy array conversion
np_z = flat_z.clone().numpy() # using '.clone()' is a MUST
np_z

array([0.9784869 , 0.16556665, 0.15542135, 0.4204805 , 0.02224775,
       0.3266385 , 1.0168396 , 1.0057176 , 1.0236931 ], dtype=float32)

In [15]:
# numpy array to tensor conversion
flat_z = torch.from_numpy(np_z.copy()) # using '.copy()' is a MUST
flat_z

tensor([0.9785, 0.1656, 0.1554, 0.4205, 0.0222, 0.3266, 1.0168, 1.0057, 1.0237])

  
  **Tensors can be kept into GPU but numpy arrays have to remain on CPU. GPUs are generally faster.**  


In [16]:
device = False
if torch.cuda.is_available():
  device = torch.device("cuda")

In [17]:
# all operations on tensors to be done in GPU
x = torch.rand(2, 2).to(device)
y = torch.rand(2, 2).to(device)
z=x+y

print(x, '\n', y, '\n', z)

tensor([[0.6929, 0.1807],
        [0.4852, 0.2245]], device='cuda:0') 
 tensor([[0.8429, 0.1386],
        [0.6781, 0.8592]], device='cuda:0') 
 tensor([[1.5358, 0.3194],
        [1.1633, 1.0837]], device='cuda:0')


In [18]:
# numpy arrays MUST be on cpu
z = z.to('cpu')
np_z = z.clone().numpy()
np_z

array([[1.535797  , 0.31938863],
       [1.1632817 , 1.0836854 ]], dtype=float32)

# Tutorial 02 - Gradient Calculation with Autograd

In [19]:
x = torch.randn(3, requires_grad=True)
x

tensor([ 1.0975,  0.4097, -1.1443], requires_grad=True)

In [20]:
y=x+2
print(y)
z=y*y*2
print(z)
z = z.mean()
print(z) 

tensor([3.0975, 2.4097, 0.8557], grad_fn=<AddBackward0>)
tensor([19.1886, 11.6131,  1.4644], grad_fn=<MulBackward0>)
tensor(10.7554, grad_fn=<MeanBackward0>)


In [21]:
# calculating dz/dx for each element of x tensor (in this case- x1,x2,x3)
#  N.B- All but the last call to backward should have the retain_graph=True option
z.backward(retain_graph=True)
print(x.grad)

tensor([4.1300, 3.2129, 1.1409])


In [22]:
# prevent gradient tracking 
# (might be needed when updating weights during training)

x = torch.randn(3, requires_grad=True)
print(x)

# way 1
print('way 1')
y=x
y.requires_grad_(False)
print(x)

# way 2
print('way 2')
y=x.detach()
print(y)

# way 3
print('way 3')
y=x+2
print(x, '\n', y)
with torch.no_grad():
  y=x+2
  print(x, '\n', y)

tensor([ 0.5602, -1.3096,  0.5676], requires_grad=True)
way 1
tensor([ 0.5602, -1.3096,  0.5676])
way 2
tensor([ 0.5602, -1.3096,  0.5676])
way 3
tensor([ 0.5602, -1.3096,  0.5676]) 
 tensor([2.5602, 0.6904, 2.5676])
tensor([ 0.5602, -1.3096,  0.5676]) 
 tensor([2.5602, 0.6904, 2.5676])


In [23]:
# dummy training example with some weights

weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
  model_output = (weights*3).sum() # loss function... probably
  
  model_output.backward()
  print(weights.grad)

  # before next iteration or optimization step MUST empty the gradient
  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


# Tutorial 03 - Back Propagation

In [27]:
# example backpropagation for a single instance

x = torch.tensor(1.0) # input
y = torch.tensor(2.0) # actual output

w = torch.tensor(1.0, requires_grad=True) # weight i.e, learnable parameter

# forward pass
y_hat = w*x # y_hat is the prediction using linear model = w*x 
loss = (y_hat - y)**2 # loss function = squared error (generally this would be MSE)

print(loss)

# backward pass
loss.backward()
print(w.grad)

# now update weight using the gradient 
# and do forward and backward pass again

tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)
